In [2]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

import json

Loading BokehJS ...

In [3]:
userScores_simple = json.load(open("userScores_simple_r42_50_inertia.json"))
# userScores = json.load(open("userScores.json"))

In [4]:
insiders = ["AAM0658", "AJR0932", "BDV0168", "BIH0745", "BLS0678", "BTL0226", "CAH0936", "DCH0843", "EHB0824", "EHD0584", "FMG0527", "FTM0406", "GHL0460", "HJB0742", "JMB0308", "JRG0207", "KLH0596", "KPC0073", "LJR0523", "LQC0479", "MAR0955", "MAS0025", "MCF0600", "MYD0978", "PPF0435", "RAB0589", "RGG0064", "RKD0604", "TAP0551", "WDD0366", "AAF0535", "ABC0174", "AKR0057", "CCL0068", "CEJ0109", "CQW0652", "DIB0285", "DRR0162", "EDB0714", "EGD0132", "FSC0601", "HBO0413", "HXL0968", "IJM0776", "IKR0401", "IUB0565", "JJM0203", "KRL0501", "LCC0819", "MDH0580", "MOS0047", "NWT0098", "PNL0301", "PSF0133", "RAR0725", "RHL0992", "RMW0542", "TNM0961", "VSS0154", "XHW0498", "BBS0039", "BSS0369", "CCA0046", "CSC0217", "GTD0219", "JGT0221", "JLM0364", "JTM0223", "MPM0220", "MSO0222"]

def getRocPoints(userScores):
    rocPoints = []

    users = userScores.keys()

    for T in range(0,10000,10):
        anomUsers = set()
        for user in users:
            for weekScore in userScores[user]:
                if weekScore > T :
                    anomUsers.add(user)

        if len(anomUsers) == 0:
            break

        insiders_caught = 0
        for insider in insiders:
            if insider in anomUsers:
                insiders_caught += 1

        true_positive = insiders_caught/len(insiders)
        false_positive = (len(anomUsers)-insiders_caught)/len(users)
        rocPoints.append( (false_positive, true_positive) )
        
    # why is everything an iterator...
    return list(reversed(rocPoints))

In [5]:
def getAUC(rocPoints):
    auc = 0
    prev = (0,0)
    for x in rocPoints:
        auc += (x[0]-prev[0])*(prev[1]+x[1])/2
        prev = x
    return auc

In [6]:
p = figure(x_range=(0,1), y_range=(0,1.001))
p.title = "ROC Curve for Dataset 4.2"
p.xaxis.axis_label = "α"
p.yaxis.axis_label = "1-ß"

def drawRoc(rocPoints, p, c, name):

    xp = list(map(lambda x : x[0], rocPoints))
    yp = list(map(lambda x : x[1], rocPoints))

    p.line(xp,yp, line_color=c, legend=name)
    # p.scatter(xp,yp, marker="x", size=10, color=c)

p.line([0,1],[0,1],color="black")

simpleRoc = getRocPoints(userScores_simple)

drawRoc(simpleRoc, p, "blue", "Simple")

print("  Simple features AUC: ", getAUC(simpleRoc))

show(p)

  Simple features AUC:  0.7813142857142857
